In [87]:
# Import libraries
import random
import math 
import numpy as np

# Import timings
import time

# Importing Pandas to create DataFrame
import pandas as pd

import networkx as nx
import matplotlib.pyplot as plt


from collections import Counter
from collections import defaultdict
from scipy.stats import gaussian_kde


# Create empty DataFrame with specific column names & types
df = pd.DataFrame({'Best weight': pd.Series(dtype='int'),
                   'Duration': pd.Series(dtype='str'),
                   'Iterations': pd.Series(dtype='float')})
df.index.rename('Experiment', inplace=True)

In [88]:
# Define variables
num_strata = 50 # WAS 100

In [89]:
def sturges_formula(n):
    return int((1 + 3.322 * math.log10(n))*2)

In [90]:
SIMECK_TYPE=16
mask = 2 ** 16 - 1

In [91]:
def highwaylist():
    alph,bet,gam,pro = [],[],[],[]
    with open('alph.txt') as ip1, open('bet.txt') as ip2, open('gam.txt') as ip3, open('pro.txt') as ip4:
        for i1,i2,i3,i4 in zip(ip1,ip2,ip3,ip4):
            hw.append(NMTS(int(i1.strip()),int(i2.strip()),int(i3.strip()),float(i4.strip()))) 

In [92]:
"""def sample_data():

    #num_strata = 50 # WAS 100
    num_strata = sturges_formula(len(hw))
    print("Number of strata:  ", num_strata)
    # sort hw list        
    hw_sorted = sorted(hw, key=lambda x: x.dz)

    # Create stata
    strata = [hw_sorted[i::num_strata] for i in range(num_strata)]
    stratum = random.choice(strata)
    
    return stratum"""

def sample_data(hw):
    # Set the sample size as a decimal percentage
    samepl_size = 0.05
    
    # group differentials by the dz attribute values
    grouped_data = defaultdict(list)
    
    for item in hw:
        grouped_data[item.dz].append(item) # store the whole item
    
    # create an empty list
    final_nmts_list = []
    
    # Sample from each differential based on dz samples
    for dz, group in grouped_data.items():
        group_size = len(group)
        num_samples = max(1, math.ceil(group_size * samepl_size)) # Ensures at least 1 of each differential if it is too small
        
        for item in group[:num_samples]:
            nmts_instance = NMTS(dx=item.dx, dy=item.dy, dz=item.dz, wt=item.wt)
            final_nmts_list.append(nmts_instance)
    
   
    print(len(final_nmts_list))
    
    
    
    
    return final_nmts_list

In [93]:
def plot_distribution():
    #global stratum
    # plot original distribution
    dz_values = [item.dz for item in hw]
    plt.hist(dz_values, bins=50, alpha=0.9, label="Original Distribution", color="#bdc9e1") 
    
    # plot sample distribution
    sampled_dz_values = [item.dz for item in stratum]
    #plt.hist(sampled_dz_values, bins=50, alpha=0.9, label="Samples Distribution", color="#045a8d")  
    
    #Plot histogram and store output
    counts, bins, patches = plt.hist(sampled_dz_values, bins=50, alpha=0.9, label="Samples Distribution", color="#045a8d")
    
    # Get maximum bin count
    max_bin_count = max(counts)
    
    # Plot data
    plt.title("Distribution of output differentials: Original vs Stratified sample")
    plt.xlabel("Output differentials")
    plt.ylabel("Frequency")
    plt.legend(loc="upper right")
    plt.yticks([max_bin_count, 5000, 10000, 15000, 20000, 25000, 30000, 35000, 40000])
    plt.axhline(y=max_bin_count, color="r", linestyle="--", label=("Y=1000 Line"))
    
    # Generate plot
    plt.show() 

In [94]:
def mask_1(a,b,wshift):
    s1=0
    var=0
    while(s1==0):
        a1=(a<<(wshift))&mask
        b1=(b<<(wshift))&mask
        s1=a1+b1
        if(s1==0):
            var=var+1
        wshift=wshift-1
        mask1=(mask<<var)&mask
    return mask1

In [95]:

def weightAND(alpha1,beta1,gamma1): 
    s=(gamma1&(~(alpha1^beta1)))&mask
    temp=bin((alpha1^beta1)&mask)
    wt=(temp[1:].count("1"))
    if(s!=0):        
        return -200,200
    else:
        return wt,math.pow(2,-wt)

In [96]:
#weightAND(0x80,0x1,0x80)

In [97]:
#right circular shifts
def ROR(x,r):
    x = ((x << (SIMECK_TYPE - r)) + (x >> r)) & mask
    return x

#left circular shifts
def ROL(x,r):
    #print(hex(x),hex(x))
    x = ((x >> (SIMECK_TYPE - r)) + (x << r)) & mask
    #print(hex(x))
    return x

In [98]:
#x=0x8000
#r=2
#ROL(x,r)

In [99]:
#a=0x2
#b=0x82
#c=a^b
#print(hex(c))

In [100]:
def find_diff_path(st1,st0,SIMECK_ROUNDS,n):


    lp=1             
    temp_wt=0
    tempdec_list=[]
    for i in range (0,SIMECK_ROUNDS):
        #print(hex(st1),hex(st0),hex(alpha),hex(beta))
        A=st1
        B=ROL(st1,beta)
        C=ROL(st1,gamma)
        #op=A^B
        wt=-200
        while(wt==-200):
            #op=hw[random.randint(0,len(hw)-1)].dz
            #op = random.choice(stratum).dz
            op = random.choice(stratum)
            wt,wt1=weightAND(A,B,op)
        D=op^C; E=st0^D
        #print(wt)
        
        tempdec_list.append(NMTS(st1,st0,op,wt)) 
        temp_wt= temp_wt+tempdec_list[i].wt
        
        #update state with new valid output differential
        st0=st1; st1=E
    n=n+1
    
    return tempdec_list, temp_wt, n

#if(len(ddt)!=0):
#            print(hex(st1),hex(st0))
#            index=(random.randint(0,len(ddt)-1))
#            temp_op=ddt[index].dz
#           temp_wt,temp_wt1=weightAND(A,B,temp_op)
#            if(temp_wt!=-200):
#                op=temp_op
#               wt,wt1=temp_wt,temp_wt1
#        else:

In [101]:
def find_best_path(st1,st0, SIMECK_ROUNDS, wt_above, best_wt):
    #print(hex(st1),hex(st0))
    #print(hex(st1),hex(st0),'best_path')
    #using n as index value for list
    n=0
    for r in range(SIMECK_ROUNDS,0,-1):
        tempdec_list, temp_wt, n = find_diff_path(st1,st0,r, n)
        if((temp_wt+wt_above) < best_wt):
            best_wt= temp_wt+wt_above
            for i,j in zip(range(n-1,22), range(len(tempdec_list))):
                dec_list[i]=(NMTS(tempdec_list[j].dx,tempdec_list[j].dy,tempdec_list[j].dz,tempdec_list[j].wt))
        #print(n)
        if(n<SIMECK_ROUNDS):
            st1=dec_list[n].dx
            st0=dec_list[n].dy
        wt_above= wt_above+dec_list[n-1].wt
    return best_wt

In [102]:
class NMTS(object):
    """__init__() functions as the class constructor"""
    def __init__(self, dx=None, dy=None, dz=None, wt=None):
        self.dx = dx
        self.dy = dy
        self.dz = dz
        self.wt = wt

In [ ]:

row_names = {}

experiment_loops = 193

target_weight = 30

for j in range(experiment_loops):
    start_time = time.time()

    beta, gamma = 5,1
    SIMECK_ROUNDS=11

    wshift=15
    dec_list=[0]*22
    wt_above=0
    best_wt=999
    s=9999


    
    
    #highwaylist()
    hw=[]
    #nested loop to run number of time
    highwaylist()
    
    # call the sample data function to get a list of the NMTS objects
    nmts_objects = sample_data(hw)
    
    # Extract sample dz values from the NMTS objects to use as the quota stratum
    stratum = [obj.dz for obj in nmts_objects]
    
  #  if j == experiment_loops - 1:
  #      plot_distribution()
    start_time = time.time()
    ch=0
    while(best_wt >= target_weight  and ch < 10403):
    #while(best_wt>25):
        #print(hex(hw[ch].dx),hex(hw[ch].dy))
        best_wt=find_best_path(hw[ch].dx,hw[ch].dy,SIMECK_ROUNDS,wt_above,best_wt)
        ch=ch+1
        if(best_wt<s):
            s=best_wt
            print(s,ch)
            if(best_wt<33):
                print("Dec list is:")   
                for i in range(0,SIMECK_ROUNDS):    
                    print(hex(dec_list[i].dx), hex(dec_list[i].dy),hex(dec_list[i].dz),(dec_list[i].wt))

    print("Best weight is:",best_wt) 

    end_time = time.time()

    elapsed_time = end_time - start_time

    print("Elapsed time: ", elapsed_time)

    df.loc[len(df.index)] = [best_wt, elapsed_time, ch]


    row_names[j] = "Experiment " + str(j + 1)

    #print("Loop: ", j)


df = df.rename(index = row_names)   

df.loc["MEAN"] = [df["Best weight"].mean(), df["Duration"].mean(), df["Iterations"].mean()]
df.loc["MEDIAN"] = [df["Best weight"].median(), df["Duration"].median(), df["Iterations"].median()]
df.loc["MIN"] = [df["Best weight"].min(), df["Duration"].min(), df["Iterations"].min()]
df.loc["MAX"] = [df["Best weight"].max(), df["Duration"].max(), df["Iterations"].max()]

df.to_csv(r'./simeck32_stratified_sampling_2.csv', encoding='utf-8', header='true')

df

#plot_distribution()
